In [216]:
import numpy as np
from IPython.core.debugger import set_trace
import re
import difflib
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import time
import random
import pickle
from lxml.html import fromstring
from collections import Counter
from requests.packages.urllib3.util import Retry
from requests.adapters import HTTPAdapter
from requests import Session, exceptions
import sys
import pymongo
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import os
import json
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
from nltk.tokenize import sent_tokenize, word_tokenize


In [190]:
# data load
with open('/Users/rohanramesh/Documents/Insight/data_bball_writers/Attempt3_mr_scrape.pickle', 'rb') as handle:
    scrapevar = pickle.load(handle)
    
    # load writer df
with open('/Users/rohanramesh/Documents/Insight/data_bball_writers/writer_df.pickle', 'rb') as handle:
    writer_df = pickle.load(handle)

In [184]:
dbname = 'writer_feature_db'
# dbname = 'writer_db'
username = 'rohanramesh' # change this to your username
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))
con = None
con = psycopg2.connect(database = dbname, user = username)

postgres://rohanramesh@localhost/writer_feature_db
True


In [188]:
# query to load entire db into pandas
sql_query = """
SELECT * FROM writer_feature;
"""
writer_feature_df = pd.read_sql_query(sql_query,con)

In [209]:
author_webnames = writer_feature_subsection['author_list']
author_list = []
for i in author_webnames:
    idx = writer_df['website_name'] == i
#     print(i)
#     print(writer_df['Idea Text'][np.where(idx)[0]].tolist()[0])
#     break
    author_list.append(writer_df['Idea Text'][np.where(idx)[0]].tolist()[0])

In [185]:
sql_query = """
SELECT AVG("n_words") AS "avg_n_words",
         AVG("neg_sent") AS "avg_neg_sent",
         AVG("neu_sent") AS "avg_neu_sent",
         AVG("pos_sent") AS "avg_pos_sent",
         AVG("neg_sent_var") AS "avg_neg_sent_var",
         AVG("neu_sent_var") AS "avg_neu_sent_var",
         AVG("pos_sent_var") AS "avg_pos_sent_var",
        AVG("n_sentences") AS "avg_n_sentences",
        AVG("n_wordspersentence") AS "avg_n_wordspersentence",
        AVG("n_wordspersent_variability") AS "avg_n_wordspersent_variability",
        AVG("wordlength") AS "avg_wordlength",
        AVG("wordlength_var") AS "avg_wordlength_var",
        AVG("wordlength_skew") AS "avg_wordlength_skew",
        "author_list"
FROM writer_feature
WHERE "n_words" > 100
GROUP BY "author_list"
HAVING COUNT(*) > 25 
ORDER BY AVG("n_words") DESC
"""
writer_feature_subsection = pd.read_sql_query(sql_query,con)
writer_feature_subsection.head(10)
writer_feature_subsection.replace([np.inf, -np.inf], np.nan)
writer_feature_subsection = writer_feature_subsection.dropna(axis=0, how='any')

In [187]:
np.shape(writer_feature_subsection)

(90, 14)

In [100]:
# type something in search key
# new_id = url_format.format('zach', 'lowe')
# new_id = 'https://www.listennotes.com/search/?q=zach%20lowe'
page = requests.get(url_format)
curr_soup = BeautifulSoup(page.text)

In [102]:
r = curr_soup.find("div", {"class": "info ellipsis"})

In [104]:
curr_soup

<!DOCTYPE html>
<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"VwMFUV5RGwEDUVlRAgY="};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0].call(o.exports,function(e){var o=t[n][1][e];return r(o||e)},o,o.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<n.length;o++)r(n[o]);return r}({1:[function(t,e,n){function r(t){try{c.console&&console.log(t)}catch(e){}}var o,i=t("ee"),a=t(12),c={};try{o=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(c.console=!0,o.indexOf("dev")!==-1&&(c.dev=!0),o.indexOf("nr_dev")!==-1&&(c.nrDev=!0))}catch(s){}c.nrDev&&i.on("internal-error",function(t){r(t.stack)}),c.dev&&i.on("fn-err",function(t,e,n){r(n.stack)}),c.dev&&(r("NR AGENT IN DEVELOPMENT MODE"),r("flags: "+a(c,function(t,e){return t}).join(", ")))}

In [254]:
top_podcasts = {}
for curr_author in author_list:
    top_podcasts[curr_author] = []
    r = word_tokenize(curr_author)
    page_idx = 0
    while page_idx < 50:
#         url = ("https://listennotes.p.mashape.com/api"
#             "/v1/search?offset={}&q={}+{}&sort_by_date=0&type=episode".format(page_idx,r[0], r[1]))
#         response = requests.get(url,
#           headers={
#             "X-Mashape-Key": "JwHAxYJ4eKmshUhN2Z7c6wf653Tsp1Q5Ug0jsnYBp1DONOnaVX",
#             "Accept": "application/json"
#           }
#         )
        a = response.content
        b = json.loads(a)
        top_podcasts[curr_author].append(b)
        page_idx += 10
        time.sleep(random.uniform(0.5,2))
        print(url)


https://listennotes.p.mashape.com/api/v1/search?offset=0&q=Steve+Kyler&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=10&q=Steve+Kyler&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=20&q=Steve+Kyler&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=30&q=Steve+Kyler&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=40&q=Steve+Kyler&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=0&q=Chris+Ballard&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=10&q=Chris+Ballard&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=20&q=Chris+Ballard&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=30&q=Chris+Ballard&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=40&q=Chris+Ballard&sort_by_date=0&type=

https://listennotes.p.mashape.com/api/v1/search?offset=10&q=Mark+Deeks&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=20&q=Mark+Deeks&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=30&q=Mark+Deeks&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=40&q=Mark+Deeks&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=0&q=Rob+Mahoney&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=10&q=Rob+Mahoney&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=20&q=Rob+Mahoney&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=30&q=Rob+Mahoney&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=40&q=Rob+Mahoney&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=0&q=Dan+Devine&sort_by_date=0&type=episode
https:/

https://listennotes.p.mashape.com/api/v1/search?offset=30&q=Howard+Beck&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=40&q=Howard+Beck&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=0&q=Scott+Howard-Cooper&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=10&q=Scott+Howard-Cooper&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=20&q=Scott+Howard-Cooper&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=30&q=Scott+Howard-Cooper&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=40&q=Scott+Howard-Cooper&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=0&q=Chris+Mannix&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=10&q=Chris+Mannix&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=20&q=

https://listennotes.p.mashape.com/api/v1/search?offset=40&q=Tom+Ziller&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=0&q=Rory+Masterson&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=10&q=Rory+Masterson&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=20&q=Rory+Masterson&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=30&q=Rory+Masterson&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=40&q=Rory+Masterson&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=0&q=Brian+Windhorst&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=10&q=Brian+Windhorst&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=20&q=Brian+Windhorst&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=30&q=Brian+Windhorst&

https://listennotes.p.mashape.com/api/v1/search?offset=10&q=Andrew+Bernucca&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=20&q=Andrew+Bernucca&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=30&q=Andrew+Bernucca&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=40&q=Andrew+Bernucca&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=0&q=Shams+Charania&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=10&q=Shams+Charania&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=20&q=Shams+Charania&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=30&q=Shams+Charania&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=40&q=Shams+Charania&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=0&q=Sam+Amico&s

https://listennotes.p.mashape.com/api/v1/search?offset=20&q=Brady+Klopfer&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=30&q=Brady+Klopfer&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=40&q=Brady+Klopfer&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=0&q=Justin+Jett&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=10&q=Justin+Jett&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=20&q=Justin+Jett&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=30&q=Justin+Jett&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=40&q=Justin+Jett&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=0&q=Derek+Bodner&sort_by_date=0&type=episode
https://listennotes.p.mashape.com/api/v1/search?offset=10&q=Derek+Bodner&sort_by_date=0&type=ep

In [111]:
# These code snippets use an open-source library. http://unirest.io/python
response = requests.get("https://listennotes.p.mashape.com/api/v1/search?offset=0&q=zach+lowe&sort_by_date=0&type=episode",
  headers={
    "X-Mashape-Key": "JwHAxYJ4eKmshUhN2Z7c6wf653Tsp1Q5Ug0jsnYBp1DONOnaVX",
    "Accept": "application/json"
  }
)

In [179]:
response.content

b'{"took": 0.684, "results": [{"transcripts_highlighted": [" household but this delay to my vacation\\n\\n00:00:51  is <span class=\\"ln-search-highlight\\">allowed</span> me to talk about a couple teams I was kind of interested in talking about anyway and that made an interesting trade and to talk about the", " waved at a <span class=\\"ln-search-highlight\\">lower</span> number that they could find out today there was a lot of different things what they\\n\\n00:07:02  settled on it they might get a shooter is a twenty four year old starting point guard the"], "image": "https://d3sv2eduhewoas.cloudfront.net/channel/image/137f13e8339e445d9a8ac31a838b5e7f.jpg", "itunes_id": 986595124, "id": "418f6ce5778a4924b5174f6705a39537", "rss": "https://www.listennotes.com/c/r/b44cc0faed314af0886f422154d0ccd6", "title_original": "Royce Young; Chris Herring", "podcast_listennotes_url": "https://www.listennotes.com/c/b44cc0faed314af0886f422154d0ccd6/", "publisher_highlighted": "ESPN, NBA, <span class

In [229]:
# import ast
# d = ast.literal_eval(a)
# print(ast.literal_eval(a))
a = response.content
b = json.loads(a)

In [230]:
b.keys()

dict_keys(['took', 'results', 'count', 'next_offset', 'total'])

In [255]:
with open('/Users/rohanramesh/Documents/Insight/data_bball_writers/top50podcasts.pickle', 'wb') as handle:
    pickle.dump(top_podcasts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [143]:
a = curr_soup.find("p")

In [252]:
response.headers

{'CF-RAY': '45f1e414abd15783-IAD', 'Content-Encoding': 'gzip', 'Content-Type': 'application/json', 'Date': 'Mon, 24 Sep 2018 02:41:15 GMT', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'Server': 'Mashape/5.0.6', 'Set-Cookie': '__cfduid=dd8a0344666b889d35f3a162818ae39b21537756874; expires=Tue, 24-Sep-19 02:41:14 GMT; path=/; domain=.listennotes.com; HttpOnly', 'Vary': 'Cookie', 'X-Cache-Status': 'MISS', 'X-Frame-Options': 'SAMEORIGIN', 'X-RateLimit-directory-apis-Limit': '1000', 'X-RateLimit-directory-apis-Remaining': '1000', 'X-RateLimit-full-text-search-quota-Limit': '1000', 'X-RateLimit-full-text-search-quota-Remaining': '986', 'X-RateLimit-typeahead-quota-Limit': '10000', 'X-RateLimit-typeahead-quota-Remaining': '10000', 'transfer-encoding': 'chunked', 'Connection': 'keep-alive'}

In [245]:
b.keys()

dict_keys(['took', 'results', 'count', 'next_offset', 'total'])